#  <center> <span style="color:blue">    <center> </span> <center> <center><span style="color:blue">Les Réseaux de Neurones Convolutifs</span></center>


## Objectif :
objectif est  de résoudre un problème de reconnaissance d’images en utilisant les
réseaux de neurones convolutifs.

## Architecture

1. Small CNN :
- Il comprend 2 couches de convolution suivies d'activation ReLU.
- La première couche de convolution utilise 64 filtres de taille 3x3.
- La seconde couche de convolution utilise 32 filtres de taille 3x3.
- Il n'y a pas de couche de pooling ou de dropout spécifiée.
- Après une couche de mise à plat (Flatten), il y a une couche dense avec 10 neurones et une activation softmax.
- Le taux d'erreur pour ce modèle est de 1.39%.
2.Medium CNN :
- Ce modèle commence également par une couche de convolution avec 32 filtres de taille 5x5 et activation ReLU.
- Il incorpore une couche de max-pooling de 2x2.
- Ensuite, il ajoute une couche de dropout avec un taux de 0.2 pour aider à prévenir le surapprentissage.
- Après la mise à plat des données, il y a une couche dense avec 128 neurones et activation ReLU, suivie d'une couche dense finale avec 10 sorties et softmax.
- Son taux d'erreur est réduit à 1.03%.
3.Large CNN :
- Il dispose de 2 couches de convolution, la première avec 30 filtres de taille 5x5 et la seconde avec 15 filtres de taille 3x3, toutes deux avec activation ReLU.
- Entre ces deux couches de convolution, il y a une couche de max-pooling de 2x2.
- Une couche de dropout avec un taux de 0.2 est également utilisée.
- Pour les couches denses, il y a une suite de trois couches avec respectivement 128, 50, et 10 neurones, les deux premières avec activation ReLU et la dernière avec softmax.
- Le taux d'erreur pour le Large CNN est de 0.82%, ce qui est le meilleur des trois modèles présentés.

## Construction du 1er réseau

#### a. Importer les données

In [1]:
from tensorflow.keras.utils import to_categorical


C:\Users\Dell\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical  # Updated line
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

K.set_image_data_format('channels_first')

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)


In [3]:
#load data
(X_train, y_train),(X_test, y_test)= mnist.load_data()

In [4]:
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28,28).astype
('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype
('float32')

'float32'

In [5]:
from tensorflow.keras.utils import to_categorical

# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]


In [11]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

def get_data_mnist():
    # Load the MNIST dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # Reshape the data to fit the model input configuration
    # Assuming 'channels_first' data format
    X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

    # Normalize the pixel values to 0-1
    X_train /= 255
    X_test /= 255

    # Apply one-hot encoding to the labels
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    num_classes = y_test.shape[1]

    return (X_train, y_train), (X_test, y_test), num_classes


In [8]:
import matplotlib.pyplot as plt

# Supposons que X_train est déjà chargé et formaté en (nombre_échantillons, 1, 28, 28) pour des images MNIST
# Convertir l'image en format 28x28 pour l'affichage
image_to_display = X_train[1].reshape(28, 28)  # La deuxième image, indice 1

# Afficher l'image
plt.imshow(image_to_display, cmap='gray')  # Utilisez cmap='gray' pour les images en niveaux de gris
plt.title("Displayed Image at X_train[1]")
plt.colorbar()  # Optionnel, ajoute une barre de couleur pour indiquer l'échelle des valeurs de pixel
plt.show()


<built-in method astype of numpy.ndarray object at 0x000001053E9D2F30>


#### b. Construire le modèle de CNN

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

def small_model(num_classes):
    # Création du modèle séquentiel
    model = Sequential()
    
    # Ajout de la première couche convolutive avec activation ReLU
    model.add(Conv2D(64, (3, 3), input_shape=(1, 28, 28), activation='relu'))
    
    # Ajout de la deuxième couche convolutive avec activation ReLU
    model.add(Conv2D(32, (3, 3), activation='relu'))
    
    # Ajout d'une couche pour aplatir les données en un seul vecteur
    model.add(Flatten())
    
    # Ajout d'une couche dense pour la classification finale avec activation softmax
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compilation du modèle
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model




#### c. Entrainer et évaluer notre Small CNN

In [10]:
# Charger les données
(X_train, y_train), (X_test, y_test), num_classes = get_data_mnist()

# Construire le modèle
model = small_model(num_classes)

# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=200,
    verbose=1
)

# Évaluer le modèle
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



Epoch 1/10


300/300 [==============================] - 49s 161ms/step - loss: 0.7715 - accuracy: 0.9296 - val_loss: 0.0876 - val_accuracy: 0.9723
Epoch 2/10
300/300 [==============================] - 52s 172ms/step - loss: 0.0582 - accuracy: 0.9820 - val_loss: 0.0714 - val_accuracy: 0.9786
Epoch 3/10
300/300 [==============================] - 52s 174ms/step - loss: 0.0350 - accuracy: 0.9886 - val_loss: 0.0664 - val_accuracy: 0.9811
Epoch 4/10
300/300 [==============================] - 54s 179ms/step - loss: 0.0243 - accuracy: 0.9920 - val_loss: 0.0929 - val_accuracy: 0.9783
Epoch 5/10
300/300 [==============================] - 55s 184ms/step - loss: 0.0182 - accuracy: 0.9942 - val_loss: 0.1041 - val_accuracy: 0.9752
Epoch 6/10
300/300 [==============================] - 55s 183ms/step - loss: 0.0195 - accuracy: 0.9938 - val_loss: 0.1253 - val_accuracy: 0.9749
Epoch 7/10
300/300 [==============================] - 55s 184ms/step - loss: 0.0176 - accuracy: 0.9939 - val_loss: 0.1015 - val

## Construction des 3 modèles après normalisation de données

### 1.Small CNN

In [12]:
# Charger les données
(X_train, y_train), (X_test, y_test), num_classes = get_data_mnist()

# Construire le modèle
model = small_model(num_classes)

# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=200,
    verbose=1
)

# Évaluer le modèle
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/10
300/300 [==============================] - 49s 163ms/step - loss: 0.2242 - accuracy: 0.9366 - val_loss: 0.0723 - val_accuracy: 0.9769
Epoch 2/10
300/300 [==============================] - 54s 180ms/step - loss: 0.0630 - accuracy: 0.9810 - val_loss: 0.0499 - val_accuracy: 0.9841
Epoch 3/10
300/300 [==============================] - 54s 181ms/step - loss: 0.0443 - accuracy: 0.9866 - val_loss: 0.0473 - val_accuracy: 0.9851
Epoch 4/10
300/300 [==============================] - 55s 184ms/step - loss: 0.0348 - accuracy: 0.9895 - val_loss: 0.0512 - val_accuracy: 0.9844
Epoch 5/10
300/300 [==============================] - 56s 188ms/step - loss: 0.0269 - accuracy: 0.9916 - val_loss: 0.0414 - val_accuracy: 0.9880
Epoch 6/10
300/300 [==============================] - 56s 186ms/step - loss: 0.0198 - accuracy: 0.9937 - val_loss: 0.0457 - val_accuracy: 0.9871
Epoch 7/10
300/300 [==============================] - 55s 182ms/step - loss: 0.0169 - accuracy: 0.9942 - val_loss: 0.0490 - val_ac

Small CNN :
*  Sur l'ensemble de formation (train), le modèle a une perte (loss) de 0.0095 et une précision (accuracy) de 99.68%.
* Sur l'ensemble de validation (val), la perte est de 0.0590 et la précision est de 98.55%.
* Sur l'ensemble de test, la perte est de 0.0590 et la précision est de 98.55%.

### 2. Medium_CNN

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

def medium_model(input_shape, num_classes):
    model = Sequential()
    
    # Convolutional layer with 32 filters of size 5x5 and ReLU activation
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    
    # MaxPooling layer with pool size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Dropout layer with 20% dropout rate
    model.add(Dropout(0.2))
    
    # Flatten layer to convert 2D data to 1D
    model.add(Flatten())
    
    # Dense layer with 128 neurons and ReLU activation
    model.add(Dense(128, activation='relu'))
    
    # Final Dense layer with 10 outputs and softmax activation for classification
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model with adam optimizer and categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Assuming input_shape and num_classes are already defined
model = medium_model((1, 28, 28), 10)

# Load and preprocess data
(X_train, y_train), (X_test, y_test), num_classes = get_data_mnist()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=200,
    verbose=1
)

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




Epoch 1/10
300/300 [==============================] - 9s 30ms/step - loss: 0.2398 - accuracy: 0.9309 - val_loss: 0.0791 - val_accuracy: 0.9779
Epoch 2/10
300/300 [==============================] - 9s 30ms/step - loss: 0.0745 - accuracy: 0.9778 - val_loss: 0.0468 - val_accuracy: 0.9859
Epoch 3/10
300/300 [==============================] - 9s 31ms/step - loss: 0.0516 - accuracy: 0.9845 - val_loss: 0.0446 - val_accuracy: 0.9862
Epoch 4/10
300/300 [==============================] - 10s 33ms/step - loss: 0.0408 - accuracy: 0.9873 - val_loss: 0.0351 - val_accuracy: 0.9883
Epoch 5/10
300/300 [==============================] - 10s 32ms/step - loss: 0.0325 - accuracy: 0.9900 - val_loss: 0.0333 - val_accuracy: 0.9889
Epoch 6/10
300/300 [==============================] - 9s 31ms/step - loss: 0.0259 - accuracy: 0.9921 - val_loss: 0.0328 - val_accuracy: 0.9893
Epoch 7/10
300/300 [==============================] - 9s 31ms/step - loss: 0.0221 - accuracy: 0.9929 - val_loss: 0.0312 - val_accuracy: 0.

Medium CNN :
* Sur l'ensemble de formation, le modèle a une perte de 0.0148 et une précision de 99.51%.
* Sur l'ensemble de validation, la perte est de 0.0303 et la précision est de 98.97%.
* Sur l'ensemble de test, la perte est de 0.0303 et la précision est de 98.97%.

### 3. Large_CNN

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

def large_model(input_shape, num_classes):
    model = Sequential()
    
    # Première couche de convolution avec 30 filtres de 5x5 et une activation ReLU
    model.add(Conv2D(30, (5, 5), activation='relu', input_shape=input_shape))
    
    # MaxPooling avec un pool size de 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Deuxième couche de convolution avec 15 filtres de 3x3 et une activation ReLU
    model.add(Conv2D(15, (3, 3), activation='relu'))
    
    # Dropout avec un taux de 20%
    model.add(Dropout(0.2))
    
    # Flatten pour aplatir les sorties en un vecteur unique
    model.add(Flatten())
    
    # Première couche Dense avec 128 neurones et activation ReLU
    model.add(Dense(128, activation='relu'))
    
    # Deuxième couche Dense avec 50 neurones et activation ReLU
    model.add(Dense(50, activation='relu'))
    
    # Couche de sortie Dense avec 10 neurones (pour les 10 classes de MNIST) et softmax
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compilation du modèle
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Assume the input_shape and num_classes are already defined
model = large_model((1, 28, 28), 10)

# Load and preprocess data
(X_train, y_train), (X_test, y_test), num_classes = get_data_mnist()

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=200,
    verbose=1
)

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
300/300 [==============================] - 12s 37ms/step - loss: 0.3033 - accuracy: 0.9073 - val_loss: 0.0644 - val_accuracy: 0.9800
Epoch 2/10
300/300 [==============================] - 11s 35ms/step - loss: 0.0741 - accuracy: 0.9776 - val_loss: 0.0453 - val_accuracy: 0.9848
Epoch 3/10
300/300 [==============================] - 11s 36ms/step - loss: 0.0514 - accuracy: 0.9843 - val_loss: 0.0353 - val_accuracy: 0.9880
Epoch 4/10
300/300 [==============================] - 11s 35ms/step - loss: 0.0414 - accuracy: 0.9869 - val_loss: 0.0344 - val_accuracy: 0.9885
Epoch 5/10
300/300 [==============================] - 11s 35ms/step - loss: 0.0328 - accuracy: 0.9898 - val_loss: 0.0302 - val_accuracy: 0.9905
Epoch 6/10
300/300 [==============================] - 11s 35ms/step - loss: 0.0284 - accuracy: 0.9916 - val_loss: 0.0266 - val_accuracy: 0.9907
Epoch 7/10
300/300 [==============================] - 11s 36ms/step - loss: 0.0241 - accuracy: 0.9923 - val_loss: 0.0261 - val_accuracy:

Large CNN :
* Sur l'ensemble de formation, le modèle a une perte de 0.0167 et une précision de 99.46%.
* Sur l'ensemble de validation, la perte est de 0.0252 et la précision est de 99.19%.
* Sur l'ensemble de test, la perte est de 0.0252 et la précision est de 99.19%.

In [18]:
large=model

## Sauvegarde et chargements des modèles Keras 

#### Fonction de Sauvegarde

In [15]:
from tensorflow.keras.models import model_from_json

def save_keras_model(model, filename):
    # Serialize model to JSON
    model_json = model.to_json()
    with open(filename + ".json", "w") as json_file:
        json_file.write(model_json)
    
    # Serialize weights to HDF5
    model.save_weights(filename + ".h5")
    print("Saved model to disk")


#### Fonction de Chargement

In [16]:
from tensorflow.keras.models import model_from_json

def load_keras_model(filename):
    # Load the architecture of the model from a json file
    with open(filename + ".json", 'r') as json_file:
        loaded_model_json = json_file.read()

    # Create model from the json data
    loaded_model = model_from_json(loaded_model_json)

    # Load weights into the new model
    loaded_model.load_weights(filename + ".h5")
    print("Loaded model from disk")

    return loaded_model


#### Compile the Model After Loading

In [19]:
save_keras_model(model, 'large')


Saved model to disk


In [21]:
loaded_model = load_keras_model('large')

Loaded model from disk


In [22]:
# Don't forget to compile the model after loading
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Use the Model

In [ ]:

# Make predictions
predictions = model.predict(X_test)


In [34]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt


def prepare_image(file_path):
    img = load_img(file_path, target_size=(28, 28), color_mode='grayscale')
    img_array = img_to_array(img)  # Convertir en tableau
    img_array = img_array / 255.0  # Normaliser entre [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Le modèle attend un lot d'images pour prédire
    return img_array

# Supposons que vous avez une nouvelle image stockée à 'chemin/vers/nouvelle_image.png'
image = prepare_image('E:/RT4/S2/DeepLearning/test77.png')


In [35]:
predictions = loaded_model.predict(image)
print("Prédictions brutes du modèle :", predictions)


1/1 [==============================] - 0s 19ms/step
Prédictions brutes du modèle : [[0.00909463 0.03985641 0.07497075 0.30186298 0.02065216 0.17897569
  0.16385247 0.06280797 0.12103488 0.02689208]]


In [36]:
predicted_class_index = np.argmax(predictions)
print("Classe prédite :", predicted_class_index)


Classe prédite : 3


## Interprétation :
* Les trois modèles montrent de très bonnes performances, avec des précisions supérieures à 98% sur les ensembles de validation et de test.
* Le modèle Large CNN semble légèrement surperformer les autres, avec une précision légèrement supérieure sur les ensembles de validation et de test, ainsi qu'une perte plus faible sur l'ensemble de test.
* Tous les modèles semblent bien généraliser, car les performances sur l'ensemble de test sont proches de celles sur l'ensemble de validation.

## Conclusion 
es trois modèles montrent des performances impressionnantes, avec des précisions supérieures à 98% sur les ensembles de validation et de test du MNIST. <br>
Cependant, aucun des modèles **n'atteint une précision de 100%** . Plusieurs raisons peuvent expliquer cela : 
- la complexité des données peut rendre certaines prédictions difficiles même pour des modèles sophistiqués
- la capacité limitée des modèles à capturer toutes les subtilités des données
- le risque de surapprentissage aux données d'entraînement, et l'optimisation des paramètres d'entraînement.

En résumé, une précision de 100% sur des données réelles est souvent hors de portée en raison de diverses limitations des modèles et de la complexité intrinsèque des données. L'objectif est d'obtenir des performances aussi proches que possible de la perfection tout en évitant le surapprentissage et en généralisant bien sur de nouvelles données.
